In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================

# --- IMPORTANT: Point this to the versioned folder from your last analysis run ---
# Example: "/Users/natalyagrokh/AI/ml_expressions/img_expressions/video_analysis/V3_20250714_093506"
RUN_DIRECTORY = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V3_20250714_093845"

# --- Charting Configuration ---
# Define the top N emotions you want to plot on the timeline
EMOTIONS_TO_PLOT = ['happiness', 'questioning', 'contempt', 'neutral', 'surprise']
FPS = 30 # Assumed frames per second of the source video
SMOOTHING_WINDOW_SECONDS = 2 # Use a 2-second rolling average
ROLLING_WINDOW = int(FPS * SMOOTHING_WINDOW_SECONDS)

In [3]:
# ==============================================================================
# 2. DATA LOADING & ANALYSIS
# ==============================================================================

# Loads a filtered emotion log, analyzes it, and saves summary charts.
def generate_report_visuals(run_dir):

    log_path = os.path.join(run_dir, "filtered_emotion_log.csv")
    if not os.path.exists(log_path):
        print(f"❌ Error: '{log_path}' not found. Please run the video analyzer script first.")
        return

    df = pd.read_csv(log_path)
    print(f"✅ Loaded '{log_path}' with {len(df)} relevant emotional events.")

    # --- Task 1: Aggregate Statistics ---
    print("\n--- Overall Emotion Distribution ---")
    # Use value_counts to get the percentage of frames for each predicted emotion
    emotion_distribution = df['predicted_label'].value_counts(normalize=True) * 100
    print(emotion_distribution.round(1).to_string())

    # --- Task 2: Temporal Smoothing ---
    # Identify the probability columns to be smoothed
    prob_columns = [f"prob_{label}" for label in EMOTIONS_TO_PLOT if f"prob_{label}" in df.columns]
    
    # Calculate the rolling average for these columns
    df_smoothed = df[prob_columns].rolling(window=ROLLING_WINDOW, min_periods=1).mean()
    df_smoothed['timestamp_seconds'] = df['timestamp_seconds'] # Carry over the timestamp
    print(f"\n✅ Applied a {SMOOTHING_WINDOW_SECONDS}-second rolling average to emotion probabilities.")

    # --- Task 3: Visualization ---
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # a) Pie Chart for Overall Distribution
    pie_chart_path = os.path.join(run_dir, "summary_emotion_pie_chart.png")
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.pie(emotion_distribution, labels=emotion_distribution.index, autopct='%1.1f%%', startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.title("Overall Distribution of Detected Emotions", fontsize=16)
    plt.savefig(pie_chart_path)
    plt.close()
    print(f"✅ Saved summary pie chart to: {pie_chart_path}")

    # b) Line Chart for Emotion Timeline
    timeline_path = os.path.join(run_dir, "emotion_timeline.png")
    fig2, ax2 = plt.subplots(figsize=(15, 7))
    for col in prob_columns:
        emotion_name = col.replace("prob_", "")
        ax2.plot(df_smoothed['timestamp_seconds'], df_smoothed[col], label=emotion_name.capitalize())
        
    plt.title("Smoothed Emotion Probabilities Over Time", fontsize=16)
    plt.xlabel("Time in Video (seconds)", fontsize=12)
    plt.ylabel("Smoothed Probability", fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(timeline_path)
    plt.close()
    print(f"✅ Saved emotion timeline graph to: {timeline_path}")


In [5]:
# ==============================================================================
# 3. MAIN EXECUTION BLOCK
# ==============================================================================
if __name__ == '__main__':
    
    # --- IMPORTANT: Point this to the versioned folder from your last analysis run ---
    # This is the only line you need to edit for each new run.
    RUN_DIRECTORY = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V3_20250714_093845"

    # The confusing if/else block has been removed.
    # We now directly call the function with the path defined above.
    generate_report_visuals(RUN_DIRECTORY)

✅ Loaded '/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V3_20250714_093845/filtered_emotion_log.csv' with 5152 relevant emotional events.

--- Overall Emotion Distribution ---
predicted_label
happiness      37.4
contempt       26.6
sadness        11.8
fear            6.3
neutral         6.0
surprise        4.7
questioning     4.1
disgust         2.0
anger           1.0
unknown         0.1

✅ Applied a 2-second rolling average to emotion probabilities.
✅ Saved summary pie chart to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V3_20250714_093845/summary_emotion_pie_chart.png
✅ Saved emotion timeline graph to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V3_20250714_093845/emotion_timeline.png


/var/folders/sk/1zc1_rjn51705bjl1cddydvm0000gn/T/ipykernel_97117/4181852666.py:54: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
